# Welcome to the Template Generator
## @ authors: Matteo Di Fiore
I hope you read the README file.

In [ ]:
# for Jupyter light
%pip install -q ipywidgets
%pip install -q nbformat
%pip install -q markdown

In [1]:
import nbformat
from nbformat.v4 import new_notebook, new_code_cell, new_markdown_cell
from ipywidgets import Text, Button, HTML, VBox, Layout
from subbox_module import SubBox
from IPython.display import display
import re
import pprint
import textwrap # for better print of statements like True and False and float(inf)
from datetime import datetime
import time #not essential just for the loading bar
import os # for creating folders and file paths

In [2]:
class ModelGenerator:
    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        init function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """



    
    def __init__(self):

        self.version = "Version 1.8.1 from 10.11.25 Matteo Di Fiore, New Feature: Constant feed disabled:  constant feed = None not 0"
        
        SubBox.clear_log() # clear log at the start of a new instance


        self.model_name = ""
        self.variables = {}
        self.inputs = {"Feed": dict(  # The feed is added as a default option in the inputs.
            description="Feed of # add what you are feeding",
            unit="L",
            concentration=0.0,
            plotting=dict(plot=None, stem=None, range=[None, None]),
        )}
        self.parameters = []
        self.constants = []
        self.rates = []
        self.folder_created = False
        self.with_feed_equations = False

        def restart(b):
            """Restart the entire interface"""
            SubBox.clear_log() # clear log at the start of a new instance
            self.model_name = ""
            self.variables = {}
            self.inputs["Feed"] = dict(                                         # The feed is added as a default option in the inputs.
                        description="Feed of # add what you are feeding",
                        unit="L",
                        concentration=0.0,
                        plotting=dict(plot=None, stem=None, range=[None, None]),
                    )
            self.parameters = []
            self.constants = []
            self.rates = []
            self.dif_equations = []
            self.folder_created = False
            self.with_feed_equations = False
            self.content_container.children = [self.main_box]



        # In __init__, change the restart button line to:
        self.restart_button = Button(description="Restart", button_style='warning')
        self.restart_button.on_click(restart)


        # --------------------------------
        # Title
        # --------------------------------
        main_title = HTML("<h2 style='text-align:center;'>Template Generator</h2>")

        # -------------------------------
        # Question 1: Ask Model Name description and goal
        # -------------------------------
        model_input = Text(description="Model Name:", layout=Layout(width='400px'), style={'description_width': '200px'})
        model_description_input = Text(description="Model description and assumptions:", layout=Layout(width='400px'), style={'description_width': '200px'})
        model_goal_input = Text(description="Model Goal:", layout=Layout(width='400px'), style={'description_width': '200px'})
        confirm_btn_model = Button(description="Confirm", button_style="success")
        model_back_btn = Button(description="Back", button_style="danger")

        model_box = SubBox(
            title="Model Information",
            inputs=[model_input, model_description_input, model_goal_input],
            buttons=[confirm_btn_model, model_back_btn],
            instructions="Please fill out all fields carefully. Enter the model name, description, assumptions and goal:",
            readme_section="Model Information"
        )

        # -------------------------------
        # Question 2: Ask Variables
        # -------------------------------
        var_input = Text(description="Variable:")
        var_unit_input = Text(description="Unit:")
        var_description_input = Text(description="Description:")
        var_add_btn = Button(description="Add Variable", button_style='info')
        var_finish_btn = Button(description="I'm finished", button_style='success')
        var_back_btn = Button(description="Back", button_style="danger")
        var_buttons_box = [var_add_btn, var_finish_btn, var_back_btn]

        var_box = SubBox(
            title="Variables",
            inputs=[var_input, var_unit_input, var_description_input],
            buttons= var_buttons_box,
            instructions="Enter variable names and units one by one:<br>Example Biomass, Biomass concentration and g/L",
            readme_section="Variables"
        )

        # -------------------------------
        # Question 3: Ask Inputs
        # -------------------------------

        in_input = Text(description="Input:")
        in_unit_input = Text(description="Unit:")
        in_description_input = Text(description="Description:")
        in_add_btn = Button(description="Add Input", button_style='info')
        in_finish_btn = Button(description="I'm finished", button_style='success')
        in_back_btn = Button(description="Back", button_style="danger")
        in_buttons_box = [in_add_btn, in_finish_btn, in_back_btn]


        in_box = SubBox(
                    title="Inputs",
                    inputs=[in_input, in_unit_input, in_description_input],
                    buttons= in_buttons_box,
                    instructions="The Feed is automatically added you dont need to write Feed here!<br>Enter input names one by one:<br>Example Substrate, Substrate concentration and g/L",
                    readme_section="Inputs"
                )


        # -------------------------------
        # Question 4: Ask Parameters
        # -------------------------------

        param_input = Text(description="Parameter:")
        para_unit_input = Text(description="Unit:")
        para_description_input = Text(description="Description:")
        para_add_btn = Button(description="Add Parameter", button_style='info')
        para_finish_btn = Button(description="I'm finished", button_style='success')
        para_back_btn = Button(description="Back", button_style="danger")
        para_buttons_box = [para_add_btn, para_finish_btn, para_back_btn]

        para_box = SubBox(
            title="Parameters",
            inputs=[param_input, para_unit_input, para_description_input],
            buttons= para_buttons_box,
            instructions="Enter parameter names one by one:<br>Example my_max, max. specific growth rate and 1/h",
            readme_section="Parameters"
        )


        # -------------------------------
        # Question 5: Ask Constants
        # -------------------------------

        const_input = Text(description="Constant:")
        co_unit_input = Text(description="Unit:")
        co_description_input = Text(description="Description:")
        co_add_btn = Button(description="Add Constant", button_style='info')
        co_finish_btn = Button(description="I'm finished", button_style='success')
        co_back_btn = Button(description="Back", button_style="danger")
        co_buttons_box = [co_add_btn, co_finish_btn, co_back_btn]

        const_box = SubBox(
            title="Constants",
            inputs=[const_input, co_unit_input, co_description_input],
            buttons= co_buttons_box,
            instructions="Enter constant names one by one:<br>Example Ks, Inhibition constant and g/L",
            readme_section="Constants"
        )

        # -------------------------------
        # Question 6: Ask Rates
        # -------------------------------

        rate_input = Text(description="Rates:")
        rate_unit_input = Text(description="Unit:")
        rate_description_input = Text(description="Description:")
        rate_add_btn = Button(description="Add Rate", button_style='info')
        rate_finish_btn = Button(description="I'm finished", button_style='success')
        rate_back_btn = Button(description="Back", button_style="danger")
        rate_buttons_box = [rate_add_btn, rate_finish_btn, rate_back_btn]

        rate_box = SubBox(
            title="Rates",
            inputs=[rate_input, rate_unit_input, rate_description_input],
            buttons= rate_buttons_box,
            instructions="Enter the variable for the rates you will use (they will also get plotted):<br>Example mu, specific growth rate and 1/h",
            readme_section="Rates"
        )

        # -------------------------------
        # Question 7: Ask Feed Parameters
        # -------------------------------

        submit_feed_params_btn = Button(description="Submit parameters", button_style='success', layout=Layout(width='400px'))


        constant_feed_start_input = Text(description="const. feed start (hours): (optional)", layout=Layout(width='400px'), style={'description_width': '200px'})
        feed_var_input = Text(description="Feed variable name:", layout=Layout(width='400px'), style={'description_width': '200px'}, value="F")
        cultivation_end_input = Text(description="Cultivation end (hours):", layout=Layout(width='400px'), style={'description_width': '200px'})
        feed_conc_input = Text(description="Feed concentration in g/L:", layout=Layout(width='400px'), style={'description_width': '200px'})
        febatch_start_input = Text(description="fedbatch start (hours): (optional)", layout=Layout(width='400px'), style={'description_width': '200px'})
        feed_back_btn = Button(description="Back", button_style="danger")

        feed_box = SubBox(
            title="Feed Parameters",
            inputs=[constant_feed_start_input, feed_var_input, cultivation_end_input, feed_conc_input, febatch_start_input],
            buttons=[submit_feed_params_btn, feed_back_btn],
            instructions="Please fill in the feed parameters. The Feed variable is automatically added to the inputs.",
            readme_section="Feed Parameters"
        )

        # -------------------------------
        # Do you want predefined feeding functions?
        # -------------------------------
        yes_btn_functions = Button(description="Yes", button_style="success")
        no_btn_functions = Button(description="No", button_style="danger")
        functions_buttons_box = [yes_btn_functions, no_btn_functions]
        functions_box = SubBox(
            title = "Predefined Feeding Functions",
            inputs = [],
            buttons = functions_buttons_box,
            instructions = "Do you want predefined feeding functions (recommended)?",
            readme_section = "Feeding Functions"
        )


        # -------------------------------
        # Confirm all UI
        # -------------------------------

        confirm_all_btn = Button(description="Confirm all and continue", button_style="success")
        confirm_box = SubBox(
            title="Confirm and Continue",
            inputs=[],
            buttons=[confirm_all_btn],
            instructions="If you are done with all the inputs press the button to continue to the differential equations.",
            readme_section="Confirm"
        )




        # --- Button Click Handler for model info ---

        def on_confirm_btn_model_clicked(b):
            model_name = model_input.value.strip()
            model_description = model_description_input.value.strip()
            model_goal = model_goal_input.value.strip()
            if model_name and model_description and model_goal:
                self.model_name = model_name
                self.model_description = model_description
                self.model_goal = model_goal
                model_box.log(f"You set the model name to: {self.model_name}")
                model_box.log(f"You set the model description to: {self.model_description}")
                model_box.log(f"You set the model name to: {self.model_goal}")
                model_box.ask_variables()
            else:
                model_box.log("Please enter all the inputs before confirming.")

        # --- Button Click Handlers for variables ---
        def on_var_add_clicked(b):
            var_name = var_input.value.strip()
            units = var_unit_input.value.strip()
            description = var_description_input.value.strip()
            if var_name and units and description:
                if var_name not in self.variables:
                    self.variables[var_name] = dict(
                        description=f"{description}",
                        unit = f"{units}",
                        initial_value=0.0,
                        boundaries=[None, None],
                        estimable=None,
                        weight=None,
                        plotting=dict(plot=None, range_fedbatch=None),
                        volume_related=None,
                        conversion_factor=None
                    )
                    var_box.log(f"Added variable: {var_name} {units}")
                    var_input.value = ""
                else:
                    var_box.log(f"Variable '{var_name} {units}' already added.")
            else:
                var_box.log("You can't leave anything empty. Please add a name, a description and a unit. If your variable has no unit write: - .")

        def on_var_finish_clicked(b):
            if not self.variables:
                var_box.log("Please add at least one variable.")
                return
            var_box.log(f"Final variables: {list(self.variables.keys())}")


        # --- Button Click Handlers for inputs ---
        def on_in_add_clicked(b):
            in_name = in_input.value.strip()
            unit = in_unit_input.value.strip()
            description = in_description_input.value.strip()
            if in_name and unit and description:
                self.inputs[in_name] = dict(
                        description=f"{description}",
                        unit=f"{unit}",
                        concentration=0.0,
                        plotting=dict(plot=None, stem=None, range=[None, None]),
                    )
                in_box.log(f"Added Input: {in_name}")
                in_input.value = ""
            else:
                in_box.log("You can't leave anything empty. Please add a name, a description and a unit. If your input has no unit write: - .")

        def on_in_finish_clicked(b):
            if not self.inputs:
                in_box.log("No Inputs added. Proceeding with Feed only")
            in_box.log(f"Final Inputs: {self.inputs}")

        # --- Button Click Handlers for parameters ---
        def on_para_add_clicked(b):
            param_name = param_input.value.strip()
            unit = para_unit_input.value.strip()
            description = para_description_input.value.strip()

            if param_name and unit and description:
                self.parameters.append({'name': param_name, 'unit': unit, 'description': description})
                para_box.log(f"Added Parameter: {param_name}")
                param_input.value = ""
            else:
                para_box.log("You can't leave anything empty. Please add a name, a description and a unit. If your parameter has no unit write: - .")

        def on_para_finish_clicked(b):
            if not self.parameters:
                para_box.log("proceeding without Parameters")
            else:
                para_box.log(f"Final Parameters: {self.parameters}")

        para_add_btn.on_click(on_para_add_clicked)
        para_finish_btn.on_click(on_para_finish_clicked)

        # --- Button Click Handler for constants    ---
        def on_co_add_clicked(b):
            const_name = const_input.value.strip()
            unit = co_unit_input.value.strip()
            description = co_description_input.value.strip()

            if const_name and unit and description:
                self.constants.append({'name': const_name, 'unit': unit, 'description': description})
                const_box.log(f"Added Constant: {const_name}")
                const_input.value = ""
            else:
                const_box.log("You can't leave anything empty. Please add a name, a description and a unit. If your constant has no unit write: - .")

        def on_co_finish_clicked(b):
            if not self.constants:
                const_box.log("Proceeding without Constants.")
            const_box.log(f"Final Constants: {self.constants}")

        # --- Button Click Handlers for rates ---
        def on_rate_add_clicked(b):
            rate_name = rate_input.value.strip()
            unit = rate_unit_input.value.strip()
            description = rate_description_input.value.strip()
            if rate_name and unit and description:
                self.rates.append({'name': rate_name, 'unit': unit, 'description': description})
                rate_box.log(f"Added Rate: {rate_name}")
            else:
                rate_box.log("You can't leave anything empty. Please add a name, a description and a unit. If your rate has no unit write: - .")

        def on_rate_finish_clicked(b):
            rate_box.log(f"Final Rates: {self.rates}")

        # --- Button Click Handlers for feed ---
        def on_submit_feed_params(b):

            self.feed_params = {}


            # Validate required inputs (feed var, cultivation end, exp feeding eq)
            if not feed_var_input.value.strip():
                feed_box.log("Feed variable name is required.")
                return
            if not cultivation_end_input.value.strip():
                feed_box.log("Cultivation end is required.")
                return



            else:
                self.feed_params.update({
                    'constant_feed_start': constant_feed_start_input.value.strip() if constant_feed_start_input.value.strip() else None,
                    'feed_var': feed_var_input.value.strip(),
                    'cultivation_end': cultivation_end_input.value.strip(),
                    'exp_feed_eq': "#put here your exponential Feeding equation(s)",
                    'constant_feed_feed_eq': "#put here your constant_feed equation(s)",
                    'feed_conc': feed_conc_input.value.strip(),
                    'fedbatch_start': febatch_start_input.value.strip() if febatch_start_input.value.strip() else None
                })



            feed_box.log("Parameters saved.")
            feed_box.log(self.feed_params)


        # --- Feeding functions button handlers ---
        def on_yes_functions_clicked(b):
            functions_box.log("You selected predefined feeding functions.")
            self.with_feed_equations = True

        def on_no_functions_clicked(b):
            functions_box.log("You selected no predefined feeding functions. Please add your own functions later by hand.")
            self.with_feed_equations = False

        # --- Confirm all button handler ---
        def on_confirm_all_clicked(b):
            SubBox.log("Confirm all clicked - moving to differential equations")
            self.ask_dif_equation()

        # --- Buttons Click call ---
        confirm_btn_model.on_click(on_confirm_btn_model_clicked)
        var_add_btn.on_click(on_var_add_clicked)
        var_finish_btn.on_click(on_var_finish_clicked)
        in_add_btn.on_click(on_in_add_clicked)
        in_finish_btn.on_click(on_in_finish_clicked)
        co_add_btn.on_click(on_co_add_clicked)
        co_finish_btn.on_click(on_co_finish_clicked)
        rate_add_btn.on_click(on_rate_add_clicked)
        rate_finish_btn.on_click(on_rate_finish_clicked)
        submit_feed_params_btn.on_click(on_submit_feed_params)
        yes_btn_functions.on_click(on_yes_functions_clicked)
        no_btn_functions.on_click(on_no_functions_clicked)
        confirm_all_btn.on_click(on_confirm_all_clicked)
        # --- Back Buttons ---
        model_back_btn.on_click(lambda b: [setattr(self, 'model_name', ''),
                                   setattr(self, 'model_description', ''),
                                   setattr(self, 'model_goal', ''),
                                   model_box.log("Model name cleared.")])
        var_back_btn.on_click(lambda b: self.delete_latest_entry("variables", var_box))
        in_back_btn.on_click(lambda b: self.delete_latest_entry("inputs", in_box))
        para_back_btn.on_click(lambda b: self.delete_latest_entry("parameters", para_box))
        co_back_btn.on_click(lambda b: self.delete_latest_entry("constants", const_box))
        rate_back_btn.on_click(lambda b: self.delete_latest_entry("rates", rate_box))
        feed_back_btn.on_click(lambda b: self.delete_latest_entry("feed_params", feed_box))



        # --- Combine everything ---
        self.main_box = VBox([main_title, model_box, var_box, in_box, para_box, const_box, rate_box, feed_box, functions_box, confirm_box])

        # Create a container for all content
        self.content_container = VBox([self.main_box])
        # show the main box
        display(self.content_container)

        # Show restart button separately
        display(self.restart_button)

    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                       delete latest entry function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """

    def delete_latest_entry(self, target_name, box):
        """
        Deletes the latest entry from the specified attribute by name.
        Works for dicts and lists.
        Example: self.delete_latest_entry('variables', var_box)
        """
        target = getattr(self, target_name, None)

        if target is None:
            box.log(f"No valid attribute found for: {target_name}")
            return

        if isinstance(target, dict):
            if target == {"Feed": self.inputs["Feed"]} and target_name == "inputs":
                box.log("Cannot delete the 'Feed' input. It is required.")
            elif target:
                last_key = next(reversed(target))
                removed_item = target.pop(last_key)
                box.log(f"Deleted latest entry from dict '{target_name}': {last_key} -> {removed_item}")
            else:
                box.log(f"Dictionary '{target_name}' is already empty.")
        elif isinstance(target, list):
            if target:
                removed_item = target.pop()
                box.log(f"Deleted latest entry from list '{target_name}': {removed_item}")
            else:
                box.log(f"List '{target_name}' is already empty.")
        else:
            box.log(f"Unsupported type for attribute '{target_name}': {type(target)}")





  



    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        ask dif equation function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
    
    def ask_dif_equation(self):
    
        # Create input boxes for each variable
        var_name_boxes = []
        for var in self.variables:
            var_name_box = Text(
                description=f"Var name for {var}:",
                layout=Layout(width='400px'),
                style={'description_width': '200px'}
            )
            var_name_boxes.append(var_name_box)
    
        # Submit + Back buttons
        submit_btn = Button(description="Submit", button_style="success")
        back_btn = Button(description="Back", button_style="danger")
        buttons = [submit_btn, back_btn]
    
        # Create SubBox
        self.dif_box = SubBox(
            title = "Differential Equations",
            inputs = var_name_boxes,
            buttons = buttons,
            instructions = "For each variable, enter the variable name to use in the code.<br>"
                         "The right-hand side of each differential equation must be filled later by hand.",
            readme_section = "Differential Equations"
        )
    
        # Replace content in container instead of display
        self.content_container.children = [self.dif_box]
    
        # --- Button handlers ---
        def on_submit_clicked(b):
            self.dif_equations = []
            for vn_box in var_name_boxes:
                vn_val = vn_box.value.strip()
                if not vn_val:
                    self.dif_box.log("Please fill in all variable names.")
                    return
                self.dif_equations.append({
                    'variable': vn_val,
                    'diffeq': f"# write here the RHS of the differential equation for {vn_val}"
                })
                self.dif_box.log(f"You added this Variable: ({vn_val})")
    
            # Go to next step
            self.ask_save_inputs()
    
        def on_back_clicked(b):
            # If user goes back, drop last entry and return to rates
            self.delete_latest_entry("dif_equations", self.dif_box)
    
        # Connect handlers
        submit_btn.on_click(on_submit_clicked)
        back_btn.on_click(on_back_clicked)

    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        ask save inputs function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """

    def ask_save_inputs(self):


        # Buttons
        yes_btn = Button(description="Yes", button_style="warning")
        no_btn = Button(description="No", button_style="success")
        back_btn = Button(description="Back", button_style="danger")
        buttons = [yes_btn, no_btn, back_btn]

        # Create SubBox
        self.save_inputs_box = SubBox(
            title="Saving Inputs",
            inputs=[],
            buttons=buttons,
            instructions="Do you want to save your inputs (recommended)?",
            readme_section="Save Inputs"
        )

        self.content_container.children = [self.save_inputs_box]

        # --- Button handlers ---
        def on_yes(b):
            self.save_inputs_box.log("Saving Inputs as a Notebook.")
            self.create_save_inputs_function()

        def on_no(b):
            self.save_inputs_box.log("No Inputs saved.")
            self.create_notebook()

        def on_back_clicked(b):
            # Go back to differential equations step
            self.ask_dif_equation()

        # Connect handlers
        yes_btn.on_click(on_yes)
        no_btn.on_click(on_no)
        back_btn.on_click(on_back_clicked)


    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        transform expression function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
    
    def transform_expression(self, expr):
        # Handle parameters
        for p in self.parameters:
            pname = p['name'] if isinstance(p, dict) else p
            pattern = r'\b' + re.escape(pname) + r'\b'
            expr = re.sub(pattern, f"parameters['{pname}']['value']", expr)
        
        # Handle constants
        for c in self.constants:
            cname = c['name'] if isinstance(c, dict) else c
            pattern = r'\b' + re.escape(cname) + r'\b'
            expr = re.sub(pattern, f"constants[{repr(cname)}]['value']", expr)
        
        return expr


    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        format variables function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
        
    def format_variables_dict(self):
        lines = ["variables = dict("]
        for name, props in self.variables.items():
            boundaries = props['boundaries']
            min_val = boundaries[0]
            max_val = boundaries[1]
            min_str = "np.inf" if min_val == float("inf") else min_val
            max_str = "np.inf" if max_val == float("inf") else max_val
    
            lines.append(f"    {name} = dict(")
            lines.append(f"        description = '{props['description']}',")
            lines.append(f"        unit = '[{props['unit']}]',")
            lines.append(f"        initial_value = {props['initial_value']}, # change this value!!!")
            lines.append(f"        boundaries = [{min_str}, {max_str}],")
            lines.append(f"        estimable = {props['estimable']},")
            lines.append(f"        weight = {props['weight']},")
            lines.append(f"        plotting = {props['plotting']},")
            lines.append(f"        volume_related = {props['volume_related']},")
            lines.append(f"        conversion_factor = {props['conversion_factor']}),")
        lines.append(")")
        return "\n".join(lines)

    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        format inputs function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
    
    def format_inputs_dict(self):
        lines = ["inputs = dict("]
        for name, props in self.inputs.items():
            boundaries = props['plotting']['range']
            min_val = "np.inf" if boundaries[0] == float("inf") else boundaries[0]
            max_val = "np.inf" if boundaries[1] == float("inf") else boundaries[1]
    
            lines.append(f"    {name} = dict(")
            lines.append(f"        description = '{props['description']}',")
            if name == "Feed":
                feed = self.feed_params
                feed_conc = feed.get('feed_conc')
                lines.append(f"        unit = '[{props['unit']}]', # change the unit if you have another type of Feed")
                lines.append(f"        concentration = {feed_conc},")
            else:
                lines.append(f"        unit = '[{props['unit']}]',")
                lines.append(f"        concentration = {props['concentration']},")
            lines.append(f"        plotting = {{'plot': {props['plotting']['plot']}, 'stem': {props['plotting'].get('stem', False)}, 'range': [{min_val}, {max_val}]}}),")
        lines.append(")")
        return "\n".join(lines)

    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        format parameters function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
    
    def format_parameters_dict(self):
        lines = ["parameters = dict("]
        for p in self.parameters:
            name = p['name']
            unit = p['unit']
            description = p['description']
            try:
                if p['values']:
                    vals = p['values']
                    lines.append(f"    {name} = dict(value = {vals['value']}, min = {vals['min']}, max = {vals['max']}, vary = {vals['vary']}, unit = '[{unit}]', description = '{description}'),")
            except KeyError:
                lines.append(f"    {name} = dict(value = 0.0, min = None, max = None, vary = None, unit = '[{unit}]', description = '{description}'),")
        lines.append(")")
        return "\n".join(lines)

    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        format constants function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
    
    def format_constants_dict(self):
        lines = ["constants = dict("]
        for c in self.constants:
            name = c['name']
            unit = c['unit']
            description = c['description']
            try:
                if c['value']:
                    value = c['value']
            except KeyError:
                value = 2  # default value if none provided
            
            lines.append(f"    {name} = dict(value = {value}, unit = '[{unit}]', description = '{description}'),")
        lines.append(")")
        return "\n".join(lines)




    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        format rates function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """

    
    def format_rates_dict(self):
        lines = ["rates = dict("]
        for r in self.rates:
            name = r['name']
            unit = r['unit']
            description = r['description']
            
            lines.append(f"    {name} = dict(unit = '[{unit}]', description = '{description}'),")
        lines.append(")")
        return "\n".join(lines)


        
    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        format automatic_generation function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
    def automatic_generation(self):
        lines = []
        lines.append("")
        lines.append("    '''------------------- generated variables -------------------'''\n")
        # Assign variables from state vector x by index
        for i, entry in enumerate(self.dif_equations):
            name = entry['variable']
            variable_items = list(self.variables.items())
            key, value = variable_items[i]
            description = value['description']
            lines.append(f"    {name} = x[{i}]    # {description}")

        
        
        # Assign parameters
        lines.append("")
        lines.append("    '''------------------- generated parameters -------------------'''\n")
        for p in self.parameters:
            name = p['name'] if isinstance(p, dict) else p
            description = p['description']
            t_name = self.transform_expression(name)
            lines.append(f"    {name} = {t_name}    # {description}")
        
        # Assign constants
        lines.append("")
        lines.append("    '''------------------- generated constants -------------------'''\n")
        for c in self.constants:
            name = c['name'] if isinstance(c, dict) else c
            description = c['description']
            t_c = self.transform_expression(name)
            lines.append(f"    {name} = {t_c}    # {description}")

        return lines





    
    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        format alg equations function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
        
    def format_equations(self):   
        lines = ['# Equations', f"def algebra(x, t):\n"]
        #assigns the right Variables for later use
        
        # Use automatic_generation to get variable/parameter/constant assignments
        lines.extend(self.automatic_generation())

        
        #this is an optional feature you can  use if you want something to happen when the feed starts
        lines.extend(self.format_feed(need_equations = False))

        lines.append("\n    '''------------------- put here your equations  -------------------'''\n")

        for i, entry in enumerate(self.rates):
            name = entry['name']
            description = entry['description']
            try:
                equation = entry['equation']
                lines.append(f"    {name} = {equation}    # {description}")
            except KeyError:
                lines.append(f"    {name} = # put here your equation to calculate {name} ({description})")
            
        lines.append("")
        
        lines.append("    # remember to add the variables you want to plot and need for your differential equations that are not rates")

        #return automatically the rates
        lines.append("    return {")

        rates_list = []
        for i, entry in enumerate(self.rates):
            name = entry['name']
            rates_list.append(f"    '{name}': {name},")
            
            # Insert a blank line after every 5 items
            if (i + 1) % 5 == 0:
                rates_list.append("\n")
        
        lines.append("".join(rates_list) + "\n           }")
        
        return "\n".join(lines)


    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        format time function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
    
    def format_time(self):

        
        lines = ['# Time variables']
        lines.append("t0 = 0.0 # initial time of simulation in h")
        lines.append("dt = 0.01 # steps of simulation in h")

        
        feed = self.feed_params
        fedbatch_start = feed.get('fedbatch_start')
        constant_feed_start = feed.get('constant_feed_start')
        end = feed.get('cultivation_end')
        lines.append(f"t_end = {end} # change if you want, this is the time of the end of simulation in h")
        lines.append(f"t_constant_feed = {constant_feed_start} # change if you want, this is the time of the beginning of continuos feeding in h")
        lines.append(f"t_fedbatch_start = {fedbatch_start} # change if you want, this is the time of the beginning of fedbatch in h")
        lines.append("t = np.linspace(t0, t_end, int(t_end/dt)+1)")

        
        
        return "\n".join(lines) 
        
    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        format feed options function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
    
    def format_feed_options(self):

        
        lines = ['# Feed options']
        
    
        feed = self.feed_params


        lines.append("# this value decides when the feeding starts based on the remaining Substrate")
        lines.append("substrate_limit_fedbatch = 0.01    # this value is in (g/L) change the value if you want")
        lines.append("")
        lines.append("# initializing the feeding class")
        lines.append("# you can call the feeding class with t_fedbatch_start, substrate_limit_fedbatch, t_constant_feed and any combination of them")
        lines.append("# just remember that if you use both t_fedbatch_start and substrate_limit_fedbatch the feeding will start when the t_fedbatch_start is reached")
        lines.append("# if you want a fedbatch (with the feed start happening at t_fedbatch) it should be like this: feed = cfb.Feed(t_fedbatch_start = t_fedbatch_start), just remember to define t_fedbatch_start with a number")
        lines.append("# if you want a fedbatch (with the feed start happening at substrate_limit_fedbatch) it should be like this: feed = cfb.Feed(substrate_limit = substrate_limit_fedbatch), just remember to attribute a number to substrate_limit_fedbatch (you can find it at the button of this code block)")
        lines.append("# if you want a fedbatch (with the feed start happening at t_fedbatch or substrate_limit_fedbatch) and a constant feed it should be like this: feed = cfb.Feed(t_const = t_constant_feed, t_fedbatch_start = t_fedbatch_start) or feed = cfb.Feed(t_const = t_constant_feed, substrate_limit = substrate_limit_fedbatch) , just remember to define teh variables with a number")
        lines.append("feed = cfb.Feed() # write here the variable which determine how you feed. In this case it is a batch simulation")
        
        return "\n".join(lines)   
        

    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        format feed function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
    
    def format_feed(self, need_equations = True):

        feed = self.feed_params

        lines = []
        
        lines.append("\n    '''------------------- feed options  -------------------'''\n")
        if "Feed" in self.inputs:
            lines.append("")
            lines.append("    # use S_feed as your feed concentration in your Formulas")
            lines.append(f"    S_feed = inputs['Feed']['concentration']")
            lines.append("") 
        
        feed_var = feed['feed_var']
        if need_equations and self.with_feed_equations:
            lines.append("    V0 = variables['Volume']['initial_value'] # import V0 as initial volume")
            #calculate F 
            exp_eq = "F0 * exp(my_set * (t - feed.ts))"
            constant_feed_eq = feed['constant_feed_feed_eq']  
        
            lines.append("    # this is an optional feature you can  use if you want some changes depending on the feeding state")
            lines.append('    if feed.batch:')
            lines.append('        # put here the things that should happen while we dont feed')
            lines.append(f'        {feed_var} = 0.0')
    
            lines.append('    if feed.feeding_first:')
            lines.append('        # put here the things that should happen when feeding starts (will happen only once)')
            lines.append('        F0 = V0 * feed.X_fedbatch_start * my_set / (Y_XS * S_feed) # change Y and S if needed')
            lines.append('        F = F0 # delete this if not needed but remember!: you can call with feed.ts the time when the feed started and with feed.X_fedbatch_start the Biomass at the moment of feed start')
            lines.append('    if feed.feeding_exp:')
            lines.append('        # this is just an example equation just write your own or change this if you want')
            lines.append("        F0 = V0 * feed.X_fedbatch_start * my_set / (Y_XS * S_feed) # change Y and S if needed")
            lines.append(f'        {feed_var} = {exp_eq}')
            lines.append('    if feed.feeding_const:')
            lines.append(f'        {feed_var} = {constant_feed_eq}')
        
        else:
            lines.append('    if feed.batch:')
            lines.append('        pass # put here the things that should happen while we dont feed')
            lines.append('    if feed.feeding_first:')
            lines.append('        pass # put here the things that should happen when feeding starts (will happen only once)')
            lines.append('        # delete this if not needed but remember!: you can call with feed.ts the time when the feed started and with feed.X_fedbatch_start the Biomass at the moment of feed start')
            lines.append('    if feed.feeding_exp:')
            lines.append('        pass # put here something that should happen while we feed')
            lines.append('    if feed.feeding_const:')
            lines.append('        pass # put here something that should happen while we induce')
            
            
        return lines

    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        format dif function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
        
    def format_dif(self):
        difs = self.dif_equations

        lines = ['# Function for the hole Model', f"def dif_simple(x, t):"]
        lines.append("\n\n")
        
        # Use automatic_generation to get variable/parameter/constant assignments
        lines.extend(self.automatic_generation())
        
        lines.append("")    
        
        # Step 2: Call alg() with all variables but after you called the feed.get
        lines.append("    '''------------------- calling feed.get function -------------------'''\n")
        lines.append("    feed.get(t, S, X)    # S and X are the variables for Substrate and Biomass change them if needed\n")

        lines.append("    '''------------------- generated rates -------------------'''\n")
        lines.append(f"    rs = algebra(x, t)")
        lines.append("")
        # Assign rates from state vector rs by index
        for i, entry in enumerate(self.rates):
            name = entry['name']
            description = entry['description']
            lines.append(f"    {name} = rs['{name}']    # {description}")
        
        lines.append("")

        # Feed 
        lines.extend(self.format_feed()) # returns the format feed with the equations and the line to call feed.get

        # Step 4: Write differential equations
        dvar_names = []  # To collect dXdt, dSdt, etc. for final dxdt
        lines.append("    '''------------------- put here your differential equations -------------------'''")
        for entry in difs:
            var = entry['variable']
            eq = entry['diffeq']
            dvardt = f"d{var}dt"

            
                
            lines.append(f"    d{var}dt = {eq}")
            dvar_names.append(dvardt)

        lines.append("\n    # this will be used by odeint")
        dxdt_list = ", ".join(dvar_names)
        lines.append(f"    dxdt = [{dxdt_list}]")
        lines.append("    return dxdt")
        
        return "\n".join(lines)

    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        format plot function for everything
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
    
    def format_plot(self):
        """
        Generates code for plotting simulation results using the sim and pe plotter.
        """
        lines = []
        lines.append(f"path_to_results, datestring, run_id = builder.init_saving(1, 1, os.path.join(os.getcwdb().decode('utf-8'), '{self.model_name + '_results'}'), None)")
        lines.append('allthedata_plot = {"1": allthedata}')
        lines.append('plot = cplt.PlotPlotly(')
        lines.append('    allthedata_plot,')
        lines.append('    path_to_results,')
        lines.append('    plot_info=f"{datestring}_{run_id}",')
        lines.append('    plot_mode="white_w_grid",')
        lines.append('    colorscheme="matlab_default",')
        lines.append('    plot_formats=("html")')
        lines.append(')')
        lines.append('plot.use_column_title = True')
        lines.append('plot.plot_by_expID(')
        lines.append('    plot_replicates=False,')
        lines.append('    show_legend=False,')
        lines.append('    plot_SE_data=False,')
        lines.append('    split_at_feed_start=False,')
        lines.append('    range_end=None,')
        lines.append('    plot_multiple_list=[')
        lines.append('        dict(members=["Biomass", "Substrate"], axes=["left", "right"])')
        lines.append('    ]')
        lines.append(')')

        return "\n".join(lines)

    
    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        format build
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
    
    def format_build(self):

        """
        generates code for building the allthedata dictionary
        """
    
        lines = []


        # Use same logic as stepy by step integration to check how to initialize the class
        lines.append("# Initializing the allthedata dictionary")
        lines.append("")
        lines.append("if feed.t_const is None and feed.t_fedbatch_start is None and feed.substrate_limit is None:")
        lines.append("    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t)")  # Example for batch only
        lines.append("elif feed.t_const is not None and feed.t_fedbatch_start is None and feed.substrate_limit is None:")
        lines.append("    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_constant_feed = t_2)")
        lines.append("elif feed.t_const is None and feed.t_fedbatch_start is not None and feed.substrate_limit is None:")
        lines.append("    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2)")
        lines.append("elif feed.t_const is None and feed.t_fedbatch_start is None and feed.substrate_limit is not None:")
        lines.append("    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2)")
        lines.append("elif feed.t_const is not None and feed.t_fedbatch_start is not None and feed.substrate_limit is None:")
        lines.append("    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2, t_3)")
        lines.append("elif feed.t_const is not None and feed.t_fedbatch_start is None and feed.substrate_limit is not None:")
        lines.append("    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2, t_3)")
        lines.append("elif feed.t_const is None and feed.t_fedbatch_start is not None and feed.substrate_limit is not None:")
        lines.append("    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2)")
        lines.append("elif feed.t_const is not None and feed.t_fedbatch_start is not None and feed.substrate_limit is not None:")
        lines.append("    builder = catd.AllthedataBuilder(variables, inputs, rates, x_and_rs, t, t_2, t_3)")
        lines.append("")
        lines.append("allthedata = builder.build()")


        return "\n".join(lines)



    
    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        format odeint function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
    
    def format_odeint(self):
        lines = []

        # Build initial conditions vector x0
        lines.append("# Define initial conditions and make a test run to estimate the feed start")
        initial_values = ", ".join(
            [f"variables['{name}']['initial_value']" for name in self.variables]
        )
        lines.append(f"x0 = [{initial_values}]")
        lines.append("") 

        lines.append("x = odeint(dif_simple, x0, t)")

        lines.append("# Print the feed start time once")
        lines.append("print('Feed started at:', feed.ts)")
        
        return "\n".join(lines)
    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        format odeint1 function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
    
    def format_odeint1(self):
        lines = []
        

        # Build phase by phase integration with odeint for different feeding strategies
        lines.append("# Actual integration phase by phase for more smoothness")
        lines.append("")
        lines.append("if feed.t_const is None and feed.t_fedbatch_start is None and feed.substrate_limit is None:")
        lines.append("    mode = 'Mode: Batch only'")  # Example for batch only
        lines.append("elif feed.t_const is not None and feed.t_fedbatch_start is None and feed.substrate_limit is None:")
        lines.append("    t_1 = np.linspace(t0, t_constant_feed, int(t_constant_feed/dt)+1)")
        lines.append("    t_2 = np.linspace(t_constant_feed, t_end, int((t_end - t_constant_feed)/dt)+1)")
        lines.append("    feed = cfb.Feed(t_const = t_constant_feed)")
        lines.append("    x_1 = odeint(dif_simple, x0, t_1)")
        lines.append("    x_2 = odeint(dif_simple, x_1[-1], t_2)")
        lines.append("    x = np.vstack((x_1[:-1], x_2))")
        lines.append("    mode = 'Mode: Constant feed only'")
        lines.append("elif feed.t_const is None and feed.t_fedbatch_start is not None and feed.substrate_limit is None:")
        lines.append("    t_1 = np.linspace(t0, t_fedbatch_start, int(t_fedbatch_start/dt)+1)")
        lines.append("    t_2 = np.linspace(t_fedbatch_start, t_end, int((t_end - t_fedbatch_start)/dt)+1)")
        lines.append("    feed = cfb.Feed(t_fedbatch_start = t_fedbatch_start)")
        lines.append("    x_1 = odeint(dif_simple, x0, t_1)")
        lines.append("    x_2 = odeint(dif_simple, x_1[-1], t_2)")
        lines.append("    x = np.vstack((x_1[:-1], x_2))")
        lines.append("    mode = 'Mode: Fedbatch only (start by time)'")
        lines.append("elif feed.t_const is None and feed.t_fedbatch_start is None and feed.substrate_limit is not None:")
        lines.append("    t_1 = np.linspace(t0, feed.ts, int(feed.ts/dt)+1)")
        lines.append("    t_2 = np.linspace(feed.ts, t_end, int((t_end - feed.ts)/dt)+1)")
        lines.append("    feed = cfb.Feed(substrate_limit = substrate_limit_fedbatch)")
        lines.append("    x_1 = odeint(dif_simple, x0, t_1)")
        lines.append("    x_2 = odeint(dif_simple, x_1[-1], t_2)")
        lines.append("    x = np.vstack((x_1[:-1], x_2))")
        lines.append("    mode = 'Mode: Fedbatch only (start by substrate limit)'")
        lines.append("elif feed.t_const is not None and feed.t_fedbatch_start is not None and feed.substrate_limit is None:")
        lines.append("    t_1 = np.linspace(t0, t_fedbatch_start, int(t_fedbatch_start/dt)+1)")
        lines.append("    t_2 = np.linspace(t_fedbatch_start, t_constant_feed, int((t_constant_feed - t_fedbatch_start)/dt)+1)")
        lines.append("    t_3 = np.linspace(t_constant_feed, t_end, int((t_end - t_constant_feed)/dt)+1)")
        lines.append("    feed = cfb.Feed(t_const = t_constant_feed, t_fedbatch_start = t_fedbatch_start)")
        lines.append("    x_1 = odeint(dif_simple, x0, t_1)")
        lines.append("    x_2 = odeint(dif_simple, x_1[-1], t_2)")
        lines.append("    x_3 = odeint(dif_simple, x_2[-1], t_3)")
        lines.append("    x = np.vstack((x_1[:-1], x_2[:-1], x_3))")
        lines.append("    mode = 'Mode: Const. feed + fedbatch (by time)'")
        lines.append("elif feed.t_const is not None and feed.t_fedbatch_start is None and feed.substrate_limit is not None:")
        lines.append("    t_1 = np.linspace(t0, feed.ts, int(feed.ts/dt)+1)")
        lines.append("    t_2 = np.linspace(feed.ts, t_constant_feed, int((t_constant_feed - feed.ts)/dt)+1)")
        lines.append("    t_3 = np.linspace(t_constant_feed, t_end, int((t_end - t_constant_feed)/dt)+1)")
        lines.append("    feed = cfb.Feed(t_const = t_constant_feed, substrate_limit = substrate_limit_fedbatch)")
        lines.append("    x_1 = odeint(dif_simple, x0, t_1)")
        lines.append("    x_2 = odeint(dif_simple, x_1[-1], t_2)")
        lines.append("    x_3 = odeint(dif_simple, x_2[-1], t_3)")
        lines.append("    x = np.vstack((x_1[:-1], x_2[:-1], x_3))")
        lines.append("    mode = 'Mode: Const. feed + fedbatch (by substrate)'")
        lines.append("elif feed.t_const is None and feed.t_fedbatch_start is not None and feed.substrate_limit is not None:")
        lines.append("    t_1 = np.linspace(t0, feed.ts, int(feed.ts/dt)+1)")
        lines.append("    t_2 = np.linspace(feed.ts, t_end, int((t_end - feed.ts)/dt)+1)")
        lines.append("    feed = cfb.Feed(t_fedbatch_start = t_fedbatch_start, substrate_limit = substrate_limit_fedbatch)")
        lines.append("    x_1 = odeint(dif_simple, x0, t_1)")
        lines.append("    x_2 = odeint(dif_simple, x_1[-1], t_2)")
        lines.append("    x = np.vstack((x_1[:-1], x_2))")
        lines.append("    mode = 'Mode: Fedbatch (start by time or substrate)'")
        lines.append("elif feed.t_const is not None and feed.t_fedbatch_start is not None and feed.substrate_limit is not None:")
        lines.append("    t_1 = np.linspace(t0, feed.ts, int(feed.ts/dt)+1)")
        lines.append("    t_2 = np.linspace(feed.ts, t_constant_feed, int((t_constant_feed - feed.ts)/dt)+1)")
        lines.append("    t_3 = np.linspace(t_constant_feed, t_end, int((t_end - t_constant_feed)/dt)+1)")
        lines.append("    feed = cfb.Feed(t_const = t_constant_feed, t_fedbatch_start = t_fedbatch_start, substrate_limit = substrate_limit_fedbatch)")
        lines.append("    x_1 = odeint(dif_simple, x0, t_1)")
        lines.append("    x_2 = odeint(dif_simple, x_1[-1], t_2)")
        lines.append("    x_3 = odeint(dif_simple, x_2[-1], t_3)")
        lines.append("    x = np.vstack((x_1[:-1], x_2[:-1], x_3))")
        lines.append("    mode = 'Mode: Const. feed + fedbatch (start by time or substrate)'")
        lines.append("print(mode)")

    
        return "\n".join(lines)

    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        format augment x function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
    def augment_x(self):
        lines = []
        
        lines.append("'''Here all the metabolic fluxes that will be plotted wil be added to x '''")

        lines.append("algebra_results = []    # just to initiate the variable")

       
    
        lines.append("")
        lines.append("# Iterate over time steps to access individual values")
        lines.append("for state_vector, time_point in zip(x, t):")
        list_names = ", ".join(
            [entry['variable'] for entry in self.dif_equations]
        )
        lines.append(f"    {list_names} = state_vector")
        lines.append("    results = algebra(state_vector, time_point)")
        lines.append("    algebra_results.append([")
        
        #append all the rates
        for i, entry in enumerate(self.rates):
            name = entry['name']
            lines.append(f"        results['{name}'],")
            
        lines.append("    ])")
        lines.append("\n# combine everything in x")
        lines.append("algebra_array = np.array(algebra_results)")
        lines.append("x_and_rs = np.hstack((x, algebra_array))")
        
        return "\n".join(lines)


    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        create save inputs function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
        
    def create_save_inputs_function(self):
        def custom_repr(obj):
            if isinstance(obj, list):
                new_list = []
                for v in obj:
                    if isinstance(v, float) and v == float('inf'):
                        new_list.append('float("inf")')
                    else:
                        new_list.append(v)
                parts = []
                for item in new_list:
                    if isinstance(item, str) and item == 'float("inf")':
                        parts.append(item)
                    else:
                        parts.append(repr(item))
                return "[" + ", ".join(parts) + "]"
            return repr(obj)
    
        def format_block(name, value):
            if name in ["variables", "inputs"]:
                lines = [f"self.{name} = {{"]
                for var, props in value.items():
                    lines.append(f"    '{var}': {{")
                    for k, v in props.items():
                        if k == 'plotting' and isinstance(v, dict):
                            lines.append(f"        '{k}': {{")
                            for pk, pv in v.items():
                                if pk == 'range' and isinstance(pv, list):
                                    # Handle float('inf') for range
                                    min_val, max_val = pv[0], pv[1]
                                    min_str = 'float("inf")' if min_val == float('inf') else repr(min_val)
                                    max_str = 'float("inf")' if max_val == float('inf') else repr(max_val)
                                    lines.append(f"            '{pk}': [{min_str}, {max_str}],")
                                else:
                                    lines.append(f"            '{pk}': {repr(pv)},")
                            lines.append("        },")
                        elif k == 'boundaries' and isinstance(v, list):
                            # If boundaries key is at this level, handle float('inf')
                            val_str = custom_repr(v)
                            lines.append(f"        '{k}': {val_str},")
                        else:
                            val_str = pprint.pformat(v, indent=8, width=80, sort_dicts=False)
                            lines.append(f"        '{k}': {val_str},")
                    lines.append("    },")
                lines.append("}")
                block = "\n".join(lines)
                return textwrap.indent(block, " " * 8) + "\n\n"
    
            elif name == "constants":
                # Handle list of dicts specially
                lines = [f"self.{name} = ["]
                for c in value:
                    if isinstance(c, dict):
                        c_line = f"    {{'name': {repr(c['name'])}, 'unit': {repr(c['unit'])}, 'description': {repr(c['description'])}}},"
                    else:
                        c_line = f"    {repr(c)},"
                    lines.append(c_line)
                lines.append("]")
                return textwrap.indent("\n".join(lines), " " * 8) + "\n\n"
    
            else:
                formatted = pprint.pformat(value, indent=4, width=80, sort_dicts=False)
                indented = textwrap.indent(f"self.{name} = {formatted}", ' ' * 8)
                return indented + "\n\n"
    
        code_lines = ["def mock_data(self):\n"]
        code_lines.append(format_block("variables", self.variables))
        code_lines.append(format_block("inputs", self.inputs))
        code_lines.append(format_block("parameters", self.parameters))
        code_lines.append(format_block("constants", self.constants))
        code_lines.append(format_block("rates", self.rates))
        code_lines.append(format_block("feed_params", self.feed_params))
        code_lines.append(format_block("dif_equations", self.dif_equations))
        code_lines.append(f"{' ' * 8}self.model_name = {repr(self.model_name)}\n\n")
        code_lines.append(f"{' ' * 8}self.model_description = {repr(self.model_description)}\n\n")
        code_lines.append(f"{' ' * 8}self.model_goal = {repr(self.model_goal)}\n\n")
        code_lines.append(f"{' ' * 8}self.with_feed_equations = {repr(self.with_feed_equations)}\n\n")
    
        full_code = "".join(code_lines)
        nb = new_notebook()
        nb.cells = [new_code_cell(full_code)]

        # --- Create a unique folder for this notebook ---
        self.folder_name = f"{self.model_name}_{datetime.now():%Y%m%d_%H%M%S}"
        os.makedirs(self.folder_name, exist_ok=True)
        self.folder_created = True
    
        filename = os.path.join(self.folder_name, f"saved_inputs_for_{self.model_name.replace(' ', '_')}.ipynb")
        with open(filename, 'w', encoding='utf-8') as f:
            nbformat.write(nb, f)
        try:
            self.save_inputs_box.log(f"Saved Inputs notebook created: {filename}")
        except AttributeError:
            pass
        self.create_notebook()
    

    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        create notebook function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
    
    def create_notebook(self):
        nb = new_notebook()

        # Title cell
        title_md = f"# Model: {self.model_name}"
        nb.cells.append(new_markdown_cell(title_md))

        # Description cell content as a multiline string
        description = (
            "###### This file has been automatically generated with the Template Generator\n"
            "###### Here is the link to the GitHub repository: https://git.tu-berlin.de/bvt-htbd/modelling/e-coli-model-overview/\n"
            f"###### {self.version}\n"
            "###### Put here the Name of the Paper\n"
            "###### Put here your DOI: 10.1021/bp9801087\n"
            f"###### Description and assumptions: {self.model_description}\n"
            f"###### Goal of Simulation: {self.model_goal}\n"
            f"###### Created on {datetime.now():%d.%m.%Y}\n"
            "###### @authors: Matteo Di Fiore, Your Name"
        )
        
        # Add the markdown cell to the notebook
        nb.cells.append(new_markdown_cell(description))

        # Install code only needed in Jupyter light
        install_code = "\n".join(["%pip install -q matplotlib", "%pip install -q plotly", "%pip install -q nbformat"])

        nb.cells.append(new_code_cell(install_code))


        
        # Import cell
        import_code = "\n".join([
            "# Used packages",
            "import numpy as np                                         # Numerical computing, arrays, linear algebra",
            "from scipy.integrate import odeint                         # ODE solver for integrating differential equations",
            "from math import exp                                       # Exponential function (for kinetics and growth rates)",
            "import class_plot as cplt                                  # Custom plotting class (data visualization)",
            "import class_fedbatch as cfb                               # Custom fed-batch model class (feeding & pulse logic)",
            "import class_allthedata as catd                            # Custom all-the-data model class (dictionary creation)",
            "from markdown_generation import generate_markdown_table    # Custom markdown generation function",
            "from IPython.display import Markdown                       # Display formatted Markdown in Jupyter notebooks",
            "import os                                                  # System coding (to save folders) ",
        ])
        nb.cells.append(new_code_cell(import_code))

        # Time cell
        time_code = self.format_time()
        nb.cells.append(new_code_cell(time_code))
        
                   
        # Feed options cell
        feed_options_code = self.format_feed_options()
        nb.cells.append(new_code_cell(feed_options_code))


        
        # Variables cell
        vars_code = self.format_variables_dict()
        nb.cells.append(new_code_cell(vars_code))

        # Variables Markdown
        vars_markdown = "\n".join([
            "# For the Variables",
            "md_output = generate_markdown_table(variables, 'Variables')",
            "# Display in Jupyter",
            "Markdown(md_output)"
        ])            
        nb.cells.append(new_code_cell(vars_markdown))

        # Inputs cell
        inputs_code = self.format_inputs_dict()
        nb.cells.append(new_code_cell(inputs_code))

        # Inputs Markdown
        inputs_markdown = "\n".join([
            "# For the Inputs",
            "md_output = generate_markdown_table(inputs, 'Inputs')",
            "# Display in Jupyter",
            "Markdown(md_output)"
        ])            
        nb.cells.append(new_code_cell(inputs_markdown))

        # Parameters cell
        params_code = self.format_parameters_dict()
        nb.cells.append(new_code_cell(params_code))

        # Parameters Markdown
        pars_markdown = "\n".join([
            "# For the Parameters",
            "md_output = generate_markdown_table(parameters, 'Parameters')",
            "# Display in Jupyter",
            "Markdown(md_output)"
        ])            
        nb.cells.append(new_code_cell(pars_markdown))

        # Constants cell
        const_code = self.format_constants_dict()
        nb.cells.append(new_code_cell(const_code))

        # Constants Markdown
        const_markdown = "\n".join([
            "# For the Constants",
            "md_output = generate_markdown_table(constants, 'Constants')",
            "# Display in Jupyter",
            "Markdown(md_output)"
        ])            
        nb.cells.append(new_code_cell(const_markdown))

        # Rates cell
        rates_code = self.format_rates_dict()
        nb.cells.append(new_code_cell(rates_code))

        # Rates Markdown
        rates_markdown = "\n".join([
            "# For the Rates",
            "md_output = generate_markdown_table(rates, 'Rates')",
            "# Display in Jupyter",
            "Markdown(md_output)"
        ])            
        nb.cells.append(new_code_cell(rates_markdown))

        # Equations cell
        eq_code = self.format_equations()
        nb.cells.append(new_code_cell(eq_code))

        #dif_equation
        dif_code = self.format_dif()
        nb.cells.append(new_code_cell(dif_code))

        #add odeint0
        odeint_code = self.format_odeint()
        nb.cells.append(new_code_cell(odeint_code))

        #add odeint1
        odeint_code1 = self.format_odeint1()
        nb.cells.append(new_code_cell(odeint_code1))

        #augment x cell
        augment_x_code = self.augment_x()
        nb.cells.append(new_code_cell(augment_x_code))


        #building allthedata dict
        build_code = self.format_build()
        nb.cells.append(new_code_cell(build_code))



        #plotting
        plot_code = self.format_plot()
        nb.cells.append(new_code_cell(plot_code))

        # check if the folder exist
        if not self.folder_created:
            # --- Create a unique folder for this notebook ---
            self.folder_name = f"{self.model_name}_{datetime.now():%Y%m%d_%H%M%S}"
            os.makedirs(self.folder_name, exist_ok=True)
    
        # Save notebook in the new folder
        notebook_path = os.path.join(self.folder_name, f"{self.model_name}.ipynb")
        with open(notebook_path, 'w', encoding='utf-8') as f:
            nbformat.write(nb, f)
    
        # Copy module files manually into the new folder
        files_to_copy = ["class_fedbatch.py", "class_plot.py", "class_allthedata.py", "markdown_generation.py"]
        for file_name in files_to_copy:
            if os.path.exists(file_name):
                with open(file_name, 'r', encoding='utf-8') as src_file:
                    content = src_file.read()
                dest_file_path = os.path.join(self.folder_name, file_name)
                with open(dest_file_path, 'w', encoding='utf-8') as dest_file:
                    dest_file.write(content)
            else:
                print(f"⚠ Warning: '{file_name}' not found, skipping copy.")

        #loading bar
        self.fake_loading_bar_simple()

        filename = f"{self.model_name}.ipynb" # saves the name of the Template for the messages

        print(f"✅ Created Template notebook '{filename}' !")
        
        try:
            self.save_inputs_box.log(f"Template notebook '{filename}' has been created successfully.")
        except AttributeError:
            pass
    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                       fake loading bar function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
    def fake_loading_bar_simple(self):
        print("Loading:", end=" ", flush=True)
        for i in range(20):
            time.sleep(0.5)  # 20 * 0.5 = 10 seconds
            print("█", end="", flush=True)
        




    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                    load  mock data from notebook function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """

    

    def load_mock_data_from_notebook(self, filename):
        """
        Loads the mock_data function from a Jupyter notebook (.ipynb) file
        and executes it to populate the instance attributes such as
        variables, inputs, parameters, constants, feed_params, and model_name.
    
        Args:
            filename (str): Path to the .ipynb file containing the mock_data function.
    
        Raises:
            ValueError: If no mock_data function is found or if execution fails.
        """
        # Read the notebook file
        nb = nbformat.read(filename, as_version=4)
        
        print(f"Loaded {len(nb.cells)} cells from {filename}")

        
        
        mock_data_code = None
        for i, cell in enumerate(nb.cells):
            if cell.cell_type == 'code':
                if 'def mock_data' in cell.source:
                    mock_data_code = cell.source
                    print("✅ Found mock_data function!")
                    break
        
        if mock_data_code is None:
            raise ValueError("No mock_data function found in the notebook")

        
        # Prepare a namespace dict to exec code safely
        namespace = {}
        
        try:
            # Execute the mock_data function code to load it into namespace
            exec(mock_data_code, namespace)
        except Exception as e:
            raise ValueError(f"Error executing mock_data function: {e}")
        
        if 'mock_data' not in namespace:
            raise ValueError("mock_data function not found after exec")
        
        try:
            # Call the loaded mock_data function with self to set attributes
            namespace['mock_data'](self)
        except Exception as e:
            raise ValueError(f"Error running mock_data function: {e}")
        
        # At this point, your instance (self) should have the attributes set
        # by the mock_data function (variables, inputs, parameters, constants, etc.)

    """
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                                                                                        mock data function
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    """
        
    def mock_data(self):
        self.variables = {
            'Biomass': {
                'description': 'Biomass',
                'unit': 'g/L',
                'initial_value': 0.1,
                'boundaries': [None, None],
                'estimable': None,
                'weight': None,
                'plotting': {'plot': None, 'range_fedbatch': None},
                'volume_related': None,
                'conversion_factor': None
            },
            'Substrate': {
                'description': 'Substrate',
                'unit': 'g/L',
                'initial_value': 13.86,
                'boundaries': [None, None],
                'estimable': None,
                'weight': None,
                'plotting': {'plot': None, 'range_fedbatch': None},
                'volume_related': None,
                'conversion_factor': None
            },
            'Acetate': {
                'description': 'Acetate',
                'unit': 'g/L',
                'initial_value': 0.0,
                'boundaries': [None, None],
                'estimable': None,
                'weight': None,
                'plotting': {'plot': None, 'range_fedbatch': None},
                'volume_related': None,
                'conversion_factor': None
            },
            'Volume': {
                'description': 'Volume',
                'unit': 'L',
                'initial_value': 7,
                'boundaries': [None, None],
                'estimable': None,
                'weight': None,
                'plotting': {'plot': None, 'range_fedbatch': None},
                'volume_related': None,
                'conversion_factor': None
            }
        }
    
        self.inputs = {
            'Feed': {
                'description': 'Feed',
                'unit': 'L',
                'concentration': 0.0,
                'plotting': {'plot': None, 'stem': None, 'range': [None, None]}
            }
        }
    
        self.parameters = [
            {
                'name': 'my_set',
                'unit': '1/h',
                'description': 'Set specific growth rate',
                'values': {'value': 0.2, 'min': None, 'max': None, 'vary': None}
                
            },
            {
                'name': 'Y_XS',
                'unit': 'g/g',
                'description': 'Yield of biomass on substrate',
                'values': {'value': 0.5, 'min': None, 'max': None, 'vary': None}
            },
            {
                'name': 'qm',
                'unit': 'g/g/h',
                'description': 'Maintenance coefficient',
                'values': {'value': 0.4, 'min': None, 'max': None, 'vary': None}
            }
        ]
    
        self.constants = [
            {'name': 'q_s_max', 'unit': 'g/(g*h)', 'description': 'Maximum substrate uptake rate'},
            {'name': 'q_o_max_mmol', 'unit': 'mmol/(g*h)', 'description': 'Maximum oxygen uptake rate in mmol'},
            {'name': 'molar_mass_O2', 'unit': 'g/mol', 'description': 'Molar mass of oxygen'},
            {'name': 'q_o_max', 'unit': 'g/(g*h)', 'description': 'Maximum oxygen uptake rate'},
            {'name': 'q_a_c_max_b', 'unit': 'g/(g*h)', 'description': 'Maximum acetate consumption rate (batch)'},
            {'name': 'q_a_c_max_f', 'unit': 'g/(g*h)', 'description': 'Maximum acetate consumption rate (fed-batch)'},
            {'name': 'K_s', 'unit': 'g/L', 'description': 'Substrate affinity constant'},
            {'name': 'K_a', 'unit': 'g/L', 'description': 'Acetate inhibition constant'},
            {'name': 'K_io', 'unit': 'g/L', 'description': 'Oxygen inhibition constant'},
            {'name': 'K_is', 'unit': 'g/L', 'description': 'Substrate inhibition constant'},
            {'name': 'Y_xs_ox', 'unit': 'g/g', 'description': 'Oxidative biomass yield on substrate'},
            {'name': 'Y_xs_of', 'unit': 'g/g', 'description': 'Overflow biomass yield on substrate'},
            {'name': 'Y_xa', 'unit': 'g/g', 'description': 'Acetate yield on biomass'},
            {'name': 'Y_os', 'unit': 'g/g', 'description': 'Oxygen yield on substrate'},
            {'name': 'Y_oa', 'unit': 'g/g', 'description': 'Oxygen yield on acetate'},
            {'name': 'Y_as', 'unit': 'g/g', 'description': 'Acetate yield on substrate'},
            {'name': 'q_m', 'unit': 'g/(g*h)', 'description': 'Maintenance rate'},
            {'name': 'C_s', 'unit': 'mol C/g', 'description': 'Carbon content in substrate'},
            {'name': 'C_a', 'unit': 'mol C/g', 'description': 'Carbon content in acetate'},
            {'name': 'C_x', 'unit': 'mol C/g', 'description': 'Carbon content in biomass'},
            {'name': 'mu_set', 'unit': '1/h', 'description': 'Set specific growth rate'},
            {'name': 'G', 'unit': '-', 'description': 'Stoichiometric coefficient'}
        ]
    
        self.rates = [
            {'name': 'q_s', 'unit': 'g/g/h', 'description': 'Specific substrate uptake rate'},
            {'name': 'q_o_s', 'unit': 'g/g/h', 'description': 'Specific oxygen uptake rate on substrate'},
            {'name': 'q_a_p', 'unit': 'g/g/h', 'description': 'Specific acetate production rate (product)'},
            {'name': 'q_a_c', 'unit': 'g/g/h', 'description': 'Specific acetate consumption rate'},
            {'name': 'q_a', 'unit': 'g/g/h', 'description': 'Total specific acetate rate'},
            {'name': 'q_a_c_en', 'unit': 'g/g/h', 'description': 'Specific acetate rate under energy limitation'},
            {'name': 'q_o', 'unit': 'g/g/h', 'description': 'Total specific oxygen uptake rate'},
            {'name': 'my', 'unit': '1/h', 'description': 'Specific growth rate'}
        ]



    
        self.dif_equations = [
            {'variable': 'X', 'diffeq': '#put here your differential equation of X'},
            {'variable': 'S', 'diffeq': '#put here your differential equation of S'},
            {'variable': 'A', 'diffeq': '#put here your differential equation of A'},
            {'variable': 'V', 'diffeq': '#put here your differential equation of V'}
        ]
    
        self.feed_params = {
            'constant_feed_start': '12',
            'feed_var': 'F',
            'cultivation_end': '20',
            'exp_feed_eq': '#put here your exponential Feeding equation(s)',
            'constant_feed_feed_eq': '#put here your constant_feed equation(s)',
            'feed_conc': '520',
            'fedbatch_start': '6'
        }
    
        self.model_name = 'Overflow_test'
        self.model_description = 'The overflow model of xu et al'
        self.model_goal = 'Example for standard use'
        self.with_feed_equations = True

       



    
    
    def example_template(self):
        self.variables = {
            'Biomass': {
                'description': 'Biomass concentration',
                'unit': 'g/L',
                'initial_value': 0.0358,
                'boundaries': [None, None],
                'estimable': None,
                'weight': None,
                'plotting': {'plot': None, 'range_fedbatch': None},
                'volume_related': None,
                'conversion_factor': None
            },
            'Substrate': {
                'description': 'Substrate',
                'unit': 'g/L',
                'initial_value': 5,
                'boundaries': [None, None],
                'estimable': None,
                'weight': None,
                'plotting': {'plot': None, 'range_fedbatch': None},
                'volume_related': None,
                'conversion_factor': None
            },
            'Volume': {
                'description': 'Volume',
                'unit': 'L',
                'initial_value': 10,
                'boundaries': [None, None],
                'estimable': None,
                'weight': None,
                'plotting': {'plot': None, 'range_fedbatch': None},
                'volume_related': None,
                'conversion_factor': None
            }
        }
    
        self.inputs = {
            'Feed': {
                'description': 'Feed volume',
                'unit': 'L',
                'concentration': 0.0,
                'plotting': {'plot': None, 'stem': None, 'range': [None, None]}
            }
        }
    
        self.parameters = [
            {
                'name': 'my_set',
                'unit': '1/h',
                'description': 'Set specific growth rate',
                'values': {'value': 0.3, 'min': None, 'max': None, 'vary': None}
            },
            {
                'name': 'qm',
                'unit': 'g/g/h',
                'description': 'Maintenance coefficient',
                'values': {'value': 0.04, 'min': None, 'max': None, 'vary': None}
            }
        ]
    
        self.constants = [
            {'name': 'Y_XS', 'value': '0.56', 'unit': 'g/g', 'description': 'Biomass yield coefficient on substrate'},
            {'name': 'qsmax', 'value': '1.6', 'unit': 'g/g/h', 'description': 'Maximum specific substrate uptake rate'},
            {'name': 'Ks', 'value': '0.05', 'unit': 'g/L', 'description': 'Saturation constant for substrate' }
        ]
    
        self.rates = [{'name': 'qs', 'unit': 'g/g', 'description': 'specific substrate uptake', 'equation': 'qsmax * S / (S + Ks)'},
                       {'name': 'my', 'unit': '1/h', 'description': 'Specific growth rate', 'equation': '(qs - qm) * Y_XS'}]

        self.dif_equations = [{'variable': 'X', 'diffeq': '(-F / V + my) * X # the equation for Biomass change'},{'variable': 'S', 'diffeq': '-F / V * (S - S_feed) - qs * X # the equation for Substrate change'}, {'variable': 'V', 'diffeq': 'F # the equation for volume change'}]
    
        self.feed_params = {
            'fedbatch_start': '6',
            'cultivation_end': '20',
            'constant_feed_start': '12',
            'feed_var': 'F',
            'exp_feed_eq': '#put here your exponential Feeding equation(s)',
            'constant_feed_feed_eq': '0.1 #put here your constant_feed equation(s)',
            'feed_conc': '500'
        }
    
        self.model_name = 'Template_Example'
        
    
        self.model_description = (
            "This is a simple template example with a working model based on Biomass, Substrate and Volume. You can expand it by adding more variables, parameters, and equations to build your own kinetic model."
        )
        self.model_goal = 'Example use for Template ready to use (already filled out). The example starts as a batch simulation'

        self.with_feed_equations = True

    
        print("Model loaded: Template_Example")
        print(self.model_description)



In [3]:
model = ModelGenerator()  # calls the class and generates the generator



Button(button_style='warning', description='Restart', style=ButtonStyle())

---
## Examples
The 2 following cells are predefined examples:

In [11]:
model.mock_data()        # This loads the dummy values
model.create_save_inputs_function() #creates a dummy template and notebook out of it

Loading: ████████████████████✅ Created Notebook 'Overflow_test.ipynb' !


In [11]:
model.example_template()
model.create_notebook()  # creates a ready to use notebook based on predefined data

Model loaded: Template_Example
This is a simple template example with a working model based on Biomass, Substrate and Volume. You can expand it by adding more variables, parameters, and equations to build your own kinetic model.
Loading: ████████████████████✅ Created Template notebook 'Template_Example.ipynb' !


---
## Using the saved inputs notebook to create a template

run this if you want to use a saved inputs notebook to create a new template based on previous inputs

In [27]:
model.load_mock_data_from_notebook('Model for Presentation_20250902_154941/Model_for_Presentation_template.ipynb') # replace with the name of your saved inputs

model.create_notebook()  # creates a notebook based on loaded data

Loaded 1 cells from Model for Presentation_20250902_154941/Model_for_Presentation_template.ipynb
✅ Found mock_data function!
Loading: ████████████████████✅ Created Notebook 'Model for Presentation.ipynb' !
